### In this notebook i gather datasets and prepare ML model

Read dataset from other notebooks

In [237]:
%store -r df_rel #other
%store -r data #prices
%store -r df  #loans
import pandas as pd
import numpy as np

no stored variable or alias #other
no stored variable or alias #prices
no stored variable or alias #loans


Time to join our dataframes. Because loan info is global for Poland I have decided not to split statistics among regions. I will group regions and find the mean of the values i df_rel and data dataframes.


In [238]:
df_rel['Mean_price'] = df_rel.mean(axis=1)

In [239]:
df_rel = df_rel['Mean_price'].to_frame()
df_rel

,Mean_price
date,
2006-10-01,593.619976
2007-01-01,682.689077
2007-04-01,645.813428
2007-07-01,-12.703526
2007-10-01,87.677613
...,...
2021-10-01,299.553089
2022-01-01,278.889970
2022-04-01,390.800201


Same for data dataframe, lets group by year and find mean of different values.

In [240]:
data

,year,building_permissions,region,flats_sold,mean_earnings
0,2005,226.0,DOLNOŚLĄSKIE,NaN,2507.77
1,2006,388.0,DOLNOŚLĄSKIE,NaN,2604.41
2,2007,605.0,DOLNOŚLĄSKIE,NaN,2847.92
3,2008,444.0,DOLNOŚLĄSKIE,NaN,3120.42
4,2009,331.0,DOLNOŚLĄSKIE,NaN,3332.46
...,...,...,...,...,...
175,2018,322.0,ŚLĄSKIE,535.0,4811.88
176,2019,264.0,ŚLĄSKIE,825.0,5206.73
177,2020,309.0,ŚLĄSKIE,795.0,5549.14
178,2021,424.0,ŚLĄSKIE,1184.0,5908.69


In [241]:
data = data.groupby('year').mean()
data

/tmp/ipykernel_7971/622920816.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby('year').mean()


,building_permissions,flats_sold,mean_earnings
year,,,
2005,124.7,NaN,2400.155
2006,196.1,NaN,2506.910
2007,273.1,NaN,2698.140
2008,244.4,NaN,2974.223
2009,161.9,NaN,3183.854
2010,164.1,583.1,3327.473
2011,167.4,504.7,3457.593
2012,159.8,697.2,3644.457
2013,129.1,855.5,3732.228


Change index type to datetime

In [242]:
data.index = pd.to_datetime(data.index, format='%Y')
data = data.reset_index()

In [243]:
#create temporary dataframes as a copy, for new quaters for each year
data2 = data.copy()
data2['year'] = data2['year'] + pd.DateOffset(months=3)
data3 = data2.copy()
data3['year'] = data3['year'] + pd.DateOffset(months=3)
data4 = data3.copy()
data4['year'] = data4['year'] + pd.DateOffset(months=3)

In [244]:
#join dataframes so we have for quaters for each year with same values
data = pd.concat([data, data2, data3, data4], ignore_index=True)

In [245]:
#set index back to years
data = data.set_index('year', drop=True)

Set index as date for df dataframe

In [246]:
df = df.set_index('Date', drop=True)

## Now we are ready to join tables.  

---


#### Because I have unified inxes, I can use simple concat.

In [247]:
print(f'df: {len(df)}')
print(f'df_rel: {len(df_rel)}')
print(f'data: {len(data)}')

df: 77
df_rel: 65
data: 72


In [248]:
df = pd.concat([df,df_rel,data], axis=1)

### Right now we have a lot of Nans. For Mean price our first values are from the 2006-10-01 so i will remove data from earlier quarters.

In [249]:
df.isnull().sum()

Loan_forcast             0
Changes_expenses         0
Alt_finansing            0
Change_economy           0
Change_rules             0
Change_criteria          0
Others                   0
Mean_price              12
building_permissions     9
flats_sold              29
mean_earnings            5
dtype: int64

In [250]:
#quick check in the middle of dataframe
df = df[df.index >= '2006-10-01']

### Flats sold column has to many Nan values, I have decided to drop it.

In [251]:
df = df.drop('flats_sold', axis=1)

#### Building permissions after additional checks looks the same as year before so i copy the data

In [253]:
df.loc['2022-01-01','building_permissions'] = 275.1
df.loc['2022-04-01','building_permissions'] = 275.1
df.loc['2022-07-01','building_permissions'] = 275.1
df.loc['2022-10-01','building_permissions'] = 275.1


In [256]:
df.isnull().sum()

Loan_forcast            0
Changes_expenses        0
Alt_finansing           0
Change_economy          0
Change_rules            0
Change_criteria         0
Others                  0
Mean_price              0
building_permissions    0
mean_earnings           0
dtype: int64

---
---
---
# THE MODEL